In [7]:
import pandas as pd
import re

In [8]:
#reading in files (have saved a version without the title analysis added (hopefully))
df_2012 = pd.read_csv(r'C:\Users\chris\Desktop\Value-Analysis-Thesis\analysis\database_miccai_2012_first_wave.csv')
df_2021 = pd.read_csv(r'C:\Users\chris\Desktop\Value-Analysis-Thesis\analysis\database_miccai_2021_first_wave.csv')

In [10]:
#removing unnecessary index column
df_2012 = df_2012.drop("Unnamed: 0", axis = 1)
df_2021 = df_2021.drop("Unnamed: 0", axis = 1)

In [46]:
#first I create a dictionary of the words in the string
#having it as a separate method means only running it once
def create_string_dic(string):
    list_of_words = string.split(" ")
    string_dictionary = {}
    for word in list_of_words:
        word = word.lower() #making all words lower case
        if word in string_dictionary: #will return true if word already is assigned a value
            string_dictionary[word] = string_dictionary.get(word) + 1
        else:
            string_dictionary[word] = 1
    return string_dictionary

In [123]:
def check_for_keywords(string_dic, keywords_dic):
    for word in keywords_dic.keys():
        if word in string_dic: #will add value if it exists, otherwise will give the keyword a 0
            keywords_dic.get(word).append(string_dic.get(word))
        else:
            keywords_dic.get(word).append(0)
         
    return keywords_dic

In [136]:
#finding the abstracts
def find_abstract(year, place):
    abstract_list = []
    with open(place, "r", encoding = 'utf-8') as part:
        article = part.read()           
        abstracts = [i.start() for i in re.finditer("Abstract", article)]
        for index in abstracts: 
            if year == 2012 and article[index+8] != "s": #removing the "Abstracts", aka the references in 2012
                abstract_list.append(article[index:index+2000])
            elif year == 2021 and article[index+9] != "T": #removing the references in 2021 ("abstract track")
                abstract_list.append(article[index:index+2000])
    return abstract_list


In [137]:
#saving the abstracts
abstract_list2012 = find_abstract(2012, r"C:\Users\chris\Desktop\Value-Analysis-Thesis\combining proceedings txt\miccai_2012_full_txt.txt")
abstract_list2021 = find_abstract(2021, r"C:\Users\chris\Desktop\Value-Analysis-Thesis\combining proceedings txt\miccai_2021_full_txt.txt")

In [ ]:
#getting the titles to combine with abstracts
list_of_dic2012 = []
titles2012 = df_2012['Title'].to_list()

    
list_of_dic2021 = []
titles2021 = df_2021['Title'].to_list()


In [155]:
#combining title and abstract
for index in range(len(titles2012)):
    titles2012[index] = titles2012[index] + " " + abstract_list2012[index]

for index in range(len(titles2021)):
    titles2021[index] = titles2021[index] + " " + abstract_list2021[index] 

In [157]:
#creating dictionary of strings to search through - only run once(technically)
for title in titles2012:
    list_of_dic2012.append(create_string_dic(title))
    
for title in titles2021:
    list_of_dic2021.append(create_string_dic(title))

In [169]:
#initialising the keyword dictionary and searching the strings for the given key words
#creating a dataframe where each column is a keyword and the value is the number of occurences of that word 
#each row corresponds to the index of the article
def generate_keyword_search_df(keywords, list_of_dic):
    initial_keywords_dic = {key:[] for key in keywords}
    for index in range(len(list_of_dic)):
        keyword_dic = (check_for_keywords(list_of_dic[index], initial_keywords_dic))   
    df = pd.DataFrame(keyword_dic) 
    return df

In [173]:
#calling the method above
keywords = ['classification', 'prediction', 'detection', 'segmentation']
df_2012_keywords = generate_keyword_search_df(keywords, list_of_dic2012)
df_2021_keywords = generate_keyword_search_df(keywords, list_of_dic2021)